# Add Traffic Signals to Network
In this case we already have a traffic signal inventory from the Georgia Department of Transportation, but code for downloading existing traffic signal data from OpenStreetMap (OSM) is also included. This code also retrieves crossings from OSM.

First, we'll find signals in the GDOT data that are not covered in the OSM data. Then we'll take these intersect the them with OSM nodes. Using the reference ID columns on the OSM links, these signals can be added to the links. A turn dataframe is then constructed with all of the links that contain at least one signal.

Once this happens, the link street name will be cross-referenced to validate the match.

- If signals are both road links, check the road name on each to see if it matches the GDOT name
- If non-road link just set it to null for now. Some of these may be crosswalks at the intersection, but they could also be walkways further away from the intersection.

This final result should be QAQC'd.

In [ ]:
from pathlib import Path 
import geopandas as gpd
import pandas as pd

from bikewaysim.paths import config
from bikewaysim.network import conflation_tools, modeling_turns

In [ ]:
links = gpd.read_file(config['network_fp']/'networks.gpkg',layer='osm_links')
nodes = gpd.read_file(config['network_fp']/'networks.gpkg',layer='osm_nodes')
#add attributes back
raw = gpd.read_file(config['osmdwnld_fp']/f"osm.gpkg",layer="raw",ignore_geometry=True)
links = pd.merge(links,raw[['osmid','highway','name']],how='left',on='osmid')
del raw
#create a name col for checking against the GDOT names
links['name0'] = links['name'].apply(lambda x: conflation_tools.contract_suffix(x))

In [ ]:
#buffer function
def buffer_signals(signal_gdf,buffer_ft):
    '''
    Use to create a copy of a gdf and buffer the point geometry
    '''
    signal_gdf = signal_gdf.copy()
    signal_gdf.geometry = signal_gdf.buffer(buffer_ft)
    return signal_gdf

# OSM Signals
No need to conflate, already embedded in the OSM network.

In [ ]:
osm_signals = gpd.read_file(config['osmdwnld_fp']/f"osm.gpkg",layer='highway_nodes')
osm_signals = osm_signals[osm_signals['highway']=='traffic_signals']
osm_signals.to_crs(config['projected_crs_epsg'],inplace=True)
osm_signal_ids = set(osm_signals['osmid'].tolist())

In [ ]:
signalized_links = links[(links['osm_A'].isin(osm_signal_ids)) | (links['osm_B'].isin(osm_signal_ids))]

## Create turn graph dataframe
_, turns_df = modeling_turns.create_pseudo_dual_graph(signalized_links,'osm_A','osm_B','osm_linkid','oneway')

#add signals ids back in
turns_df['signalized'] = turns_df['source_B'].isin(osm_signal_ids)

#drop unsignalized turn movements
turns_df = turns_df[turns_df['signalized']==True]

#export
turns_df.to_pickle(config['network_fp']/'osm_signals.pkl')

# GDOT Signals

In [ ]:
# read gdot signals
keep = ['signalID','mainStreetName','sideStreetName','geometry']
gdot_signals = gpd.read_file(config['gdot_signals_fp'],mask=gpd.read_file(config['studyarea_fp'])).to_crs(links.crs)[keep]
gdot_signals.head()

In [ ]:
import re
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName']
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName'] 
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis #TODO instead put / / instead so it can be sperated out
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: x.replace('MLK','martin luther king'))#re.sub(r'MLK', 'martin luther king', x)) # change mlk to full version
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: x.replace('MLK','martin luther king'))#re.sub(r'MLK', 'martin luther king', x)) # change mlk to full version
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: x.replace('RDA','ralph david abernathy'))#re.sub(r'RDA', 'ralph david abernathy', x)) # change mlk to full version
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: x.replace('RDA','ralph david abernathy'))# re.sub(r'RDA', 'ralph david abernathy', x)) # change mlk to full version
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'[()]', '/', x)) # remove anything in parenthesis
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'[()]', '/', x)) # remove anything in parenthesis
mainStreetName = gdot_signals['mainStreetName0'].apply(lambda x: x.split('/'))
sideStreetName = gdot_signals['sideStreetName0'].apply(lambda x: x.split('/'))
gdot_signals['gdot_names'] = mainStreetName + sideStreetName
gdot_signals['gdot_names'] = gdot_signals['gdot_names'].apply(lambda x: [y for y in x if len(y.replace(' ','')) > 0])
gdot_signals['gdot_names'] = gdot_signals['gdot_names'].apply(lambda list_of_names: [conflation_tools.contract_suffix(name) for name in list_of_names])
gdot_signals.sample(20)

# #some specific regex for the gdot dataset
# import re 

# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: x.lower())
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'sr \d+', '', x)) # remove the state routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'us \d+', '', x)) # remove the us routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'ga \d+', '', x)) # remove the ga routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'i-\d+', '', x)) # remove the us routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'phb \d+', '', x)) # remove the us routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis
# # gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub('/', ' ', x)) # replace / with space
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub('ramp', ' ', x)) # remove ramps from name

# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: x.lower())
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'sr \d+', '', x)) # remove the state routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'us \d+', '', x)) # remove the us routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'ga \d+', '', x)) # remove the ga routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'i-\d+', '', x)) # remove the us routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'phb \d+', '', x)) # remove the us routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis
# # gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub('/', ' ', x)) # replace / with space
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub('ramp', ' ', x)) # remove ramps from name

## Symmetric Difference
Find GDOT signals that are not already covered by OSM. Maybe ask an undergrad to add these into OSM later.

In [ ]:
print(gdot_signals.shape[0],'GDOT signals and',osm_signals.shape[0],'OSM signals')
buffer_ft = 100 #selecting 100 ft based on city block sizes and that's about where the number of gdot signals not represented by osm signals drops
buffered_osm_signals = buffer_signals(osm_signals,buffer_ft)
difference = gdot_signals.overlay(buffered_osm_signals,how='difference')
difference = difference[difference.drop(columns=['gdot_names']).duplicated()==False]
print('Around',difference.shape[0],'GDOT traffic signals not in OSM')

## Add these new GDOT signals into the OSM network
Set a buffer for each signal and find all candidate nodes associated
- Filter OSM nodes to only consider links with a node with a degree higher than 2 and that's labelled as a road
- Buffer and intersect the GDOT signals with the filtered OSM nodes
- Check the road names for links attached to a candidate node


we set a buffer distance around each signal to find all candidate nodes associated with the traffic signal.
- Then, we remove links that are unlikely to be signalized intersections

In [ ]:
from collections import Counter

#only consider road nodes (if there's a service road that connects to a signalized intersection this should still count them)
only_roads = links[links['link_type'].isin(['road'])].copy()
road_nodes = pd.Series(Counter(only_roads['osm_A'].tolist()+only_roads['osm_B'].tolist()))
road_nodes = set(road_nodes[road_nodes>2].index.tolist()) # and remove matches where degree is 2 or less
road_nodes = nodes[nodes['osm_N'].isin(road_nodes)]

buffer_ft = 100
gdot_buffered = buffer_signals(difference,buffer_ft)
candidate_signals = gpd.overlay(road_nodes,gdot_buffered,how="intersection")
# candidate_signals.explore()

# subset the links
candidate_signals0 = set(candidate_signals['osm_N'].tolist())
candidate_links = links[links['osm_A'].isin(candidate_signals0) | links['osm_B'].isin(candidate_signals0)].copy()
candidate_links = candidate_links[candidate_links['link_type']=='road']

In [ ]:
# add the candidate nodes info to the candidate links so that we can check street name
from importlib import reload
reload(conflation_tools)
A = candidate_links.merge(candidate_signals,left_on='osm_A',right_on='osm_N')
name_check = A.apply(lambda row: any([conflation_tools.name_check(row['name0'],name,.6) for name in row['gdot_names']]), axis = 1)
# A[name_check==False].sample(30)
A = A[name_check]
A = set(A['osm_linkid'].tolist())

B = candidate_links.merge(candidate_signals,left_on='osm_B',right_on='osm_N')
name_check = B.apply(lambda row: any([conflation_tools.name_check(row['name0'],name,.6) for name in row['gdot_names']]), axis = 1)
# main_check = B.apply(lambda row: conflation_tools.name_check(row['name'],row['mainStreetName']),axis=1)
# side_check = B.apply(lambda row: conflation_tools.name_check(row['name'],row['sideStreetName']),axis=1)
B = B[name_check]
B = set(B['osm_linkid'].tolist())

candidate_links = links[links['osm_linkid'].isin(set.union(A,B))]
# candidate_links.explore()

In [ ]:
_, turns_df = modeling_turns.create_pseudo_dual_graph(candidate_links,'osm_A','osm_B','osm_linkid','oneway')

In [ ]:
#add signals ids back in
turns_df['signalized'] = turns_df['source_B'].isin(candidate_signals0)
turns_df = turns_df[turns_df['signalized'] == True]

# turns_df.drop(columns=['source','target'],inplace=True)

turns_df.to_pickle(config['network_fp']/'gdot_signals.pkl')

# QAQC
None of the data overlap.

In [ ]:
# osm = pd.read_parquet(config['network_fp']/'osm_signals.parquet')
# gdot = pd.read_parquet(config['network_fp']/'gdot_signals.parquet')


In [ ]:
# osm.signalized.value_counts()

In [ ]:
# gdot.signalized.value_counts()

In [ ]:

# gdot.set_index(['source_linkid','source_reverse_link','target_linkid','target_reverse_link'],inplace=True)
# osm.set_index(['source_linkid','source_reverse_link','target_linkid','target_reverse_link'],inplace=True)

# gdot = gdot[['signalized','source_B']]
# gdot.columns = ['osm','osm_N']
# osm = osm[['signalized','source_B']]
# osm.columns = ['gdot','osm_N']
# node_geo = dict(zip(nodes['osm_N'],nodes.geometry))
# test = pd.merge(gdot,osm,left_index=True,right_index=True)
# test

# test[test['osm'].notna() & test['gdot'].notna()]
# test['geometry'] = test['source_B'].map(node_geo)
# test = gpd.GeoDataFrame(test,crs=config['projected_crs_epsg'])
# test
# test.explore()
# tcandidate_signals = candidate_signals[candidate_signals['osm_N'].isin(road_nodes)]
# test_dict = dict(zip(candidate_signals['osm_N'],candidate_signals['signalID']))

# links['signal_A'] = links['osm_A'].map(test_dict)
# links['signal_B'] = links['osm_B'].map(test_dict)
# candidate_signals_links = links[(links['signal_A'] != links['signal_B']) & (links['signal_A'].notna() | links['signal_B'].notna()) & (links['link_type']=='road')]
# #candidate_signals_links.explore()
# candidate_signals_links.columns
# _, turns_df = modeling_turns.create_pseudo_dual_graph(signalized_links,'osm_A','osm_B','osm_linkid','oneway')
# _, turns_df = modeling_turns.create_pseudo_dual_graph(signalized_links,'osm_A','osm_B','osm_linkid','oneway')
# #add signals ids back in
# test_dict = dict(zip(candidate_signals['osm_N'],candidate_signals['signalID']))

# turns_df['source_signal_A'] = turns_df['source_A'].map(test_dict)
# turns_df['source_signal_B'] = turns_df['source_B'].map(test_dict)
# turns_df['target_signal_A'] = turns_df['target_A'].map(test_dict)
# turns_df['target_signal_B'] = turns_df['target_B'].map(test_dict)
# #assign the source signal and target signal parts based on the link directions
# import numpy as np
# turns_df['source_signal'] = np.where(turns_df['source_reverse_link'], turns_df['source_signal_A'], turns_df['source_signal_B'])
# turns_df['target_signal'] = np.where(turns_df['target_reverse_link']==False, turns_df['source_signal_B'], turns_df['source_signal_A'])
# turns_df.drop(columns=['source_signal_A','source_signal_B','target_signal_A','target_signal_B'],inplace=True)
# #add signals ids back in
# test_dict = dict(zip(candidate_signals['osm_N'],candidate_signals['signalID']))

# turns_df['source_signal_A'] = turns_df['source_A'].map(test_dict)
# turns_df['source_signal_B'] = turns_df['source_B'].map(test_dict)
# turns_df['target_signal_A'] = turns_df['target_A'].map(test_dict)
# turns_df['target_signal_B'] = turns_df['target_B'].map(test_dict)
# #assign the source signal and target signal parts based on the link directions
# import numpy as np
# turns_df['source_signal'] = np.where(turns_df['source_reverse_link'], turns_df['source_signal_A'], turns_df['source_signal_B'])
# turns_df['target_signal'] = np.where(turns_df['target_reverse_link']==False, turns_df['source_signal_B'], turns_df['source_signal_A'])
# turns_df.drop(columns=['source_signal_A','source_signal_B','target_signal_A','target_signal_B'],inplace=True)
# #add name and the signal cross street names back
# name_dict = dict(zip(links['osm_linkid'],links['name']))

# turns_df['source_name'] = turns_df['source_linkid'].map(name_dict)
# turns_df['target_name'] = turns_df['target_linkid'].map(name_dict)
# candidate_signals
# candidate_signals_links = links[(links['signal_A'] == links['signal_B']) & (links['link_type']=='road')]
# #add name
# pd.merge(candidate_signals_links,candidate_signals.drop(columns=['geometry']),on='')
# A = pd.merge(links[['osm_A']],candidate_signals.drop(columns=['geometry']),left_on='osm_A',right_on='osm_N',how='left')
# # A.drop(columns=['osm_A','osm_N'],inplace=True)
# # A.columns = A.columns + '_A'

# B = pd.merge(links[['osm_B']],candidate_signals.drop(columns=['mainStreetName','sideStreetName','geometry']),left_on='osm_B',right_on='osm_N',how='left')
# # B.drop(columns=['osm_B','osm_N'],inplace=True)
# # B.rename(columns={'signalID':'signalID_B'},inplace=True)

# links[['osm_A']]

# B.shape[0]
# A.shape[0] 

# test = pd.concat([links,A,B],axis=1)



# test
# candidate_signals

# intersect = {key:item for key, item in intersect.items() if key in road_nodes}

# signals['buffered_geo'] = signals.buffer(buffer_ft)
# signals.set_geometry('buffered_geo',inplace=True)
# #signals.explore()
# ## Next, we intersect these bufferred signals with the street nodes
# intersect = gpd.overlay(nodes,signals,how='intersection')
# intersect.head()
# # intersect = intersect[['N','signalID']]#,'mainStreetName','sideStreetName']]
# # intersect = dict(zip(intersect['N'],intersect['signalID']))
# ## First identify public road intersections 
# Most signals should only be at the intersection of public roads (and maybe some major parking lot/service road entrances), knowing this subset to only look at public roads and then calculate the degree of the road nodes. Remove signal id matches for links with degree of 2 or less.
# links['link_type'].unique()
# only_roads = links['link_type'].isin(['road','service'])
# road_nodes = links['A'].append(links['B']).value_counts()
# #remove matches where degree is 2 or less
# road_nodes = road_nodes[road_nodes>2].index.tolist()
# intersect = {key:item for key, item in intersect.items() if key in road_nodes}

# only_roads = links['link_type']=='road'
# road_nodes = links['A'].append(links['B']).value_counts()
# #remove matches where degree is 2 or less
# road_nodes = road_nodes[road_nodes>2].index.tolist()
# intersect = {key:item for key, item in intersect.items() if key in road_nodes}
# ### With that done, we assign the signal ID to the node and add it as an attribute in links
# links['signal_A'] = links['A'].map(intersect)
# links['signal_B'] = links['B'].map(intersect)
# nodes['signalid'] = nodes['N'].map(intersect)
# Drop null values
# links = links[~links[['signal_A','signal_B']].isna().all(axis=1)]
# nodes = nodes[~nodes['N'].isna()]
# ## In the Export Network notebook, we'll process this data further
# links.to_file(network_fp/'signals_added.gpkg',layer='links')
# nodes.to_file(network_fp/'signals_added.gpkg',layer='nodes')